Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [2]:
from itertools import product
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse
import logging
import platform
from collections import Counter
from scipy.sparse import linalg, vstack, hstack, identity, eye
from functools import reduce
import copy

In [3]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [4]:
x = make_set_covering_problem(1000, 1000, .3)
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: False


In [11]:
num_points = [100, 1_000, 5_000]
num_sets = num_points
density = [0.3, 0.7]

## Hill Climbing

In [5]:
def goal_check(x,  num_pts):
    sol = [False for _ in range(num_pts)]
    for s in range(num_pts):
        for p in range(num_pts):
            sol[p] = sol[p] or x[s, p]
    
    return np.all(sol)

assert goal_check(x, 1000)

In [6]:
def fitness(sets, state):
    cost = np.sum(state)
    if np.array(state).any():
        valid = sets[np.array(state), :].max(axis=0).sum()
    else:
        valid = 0
    return valid, -cost


In [7]:
def tweak(state, size):
    new_state = state.copy()
    index = randint(0, size - 1)
    new_state[index] = not new_state[index]
    return new_state

In [8]:
def hill_climbing(problem_size, num_sets, density, nb_steps = 100000):
    sets = make_set_covering_problem(num_sets, num_sets, .3).toarray()
    initial_state = [False for _ in range(num_sets)]
    current_state = initial_state
    fit = fitness(sets, initial_state)
    print(" The fitness of the initial state is : ", fit)
    counter  = 0
    for step in range(nb_steps):
        new_state = tweak(current_state, problem_size)
        # if step % 50 == 0 : print(new_state)
        counter += 1  #maybe it should be two
        new_fit = fitness(sets, new_state)
        if fit <= new_fit:
            current_state = new_state
            fit = new_fit
            # print(fitness(sets, current_state))
    return fitness(sets, current_state), counter

In [9]:
res, count = hill_climbing(1000, 1000, 0.6)

print(" The fitness at the end of hill climbing is {} and they were {} call to the fitness function.".format(res[1], count))

 The fitness of the initial state is :  (0, 0)
 The fitness at the end of hill climbing is -15 and they were 100000 call to the fitness function.


In [14]:
res_list = []
for nb_pts in num_points:
    for dst in density : 
        res, count = hill_climbing(nb_pts, nb_pts, dst)
        print(" The fitness at the end of hill climbing for {} points and a  {} density is {} and they were {} call to the fitness function.".format(nb_pts, dst, res[1], count))
        res_list.append([nb_pts, dst, res[1], count])

res_list


 The fitness of the initial state is :  (0, 0)
 The fitness at the end of hill climbing for 100 points and a  0.3 density is -8 and they were 100000 call to the fitness function.
 The fitness of the initial state is :  (0, 0)
 The fitness at the end of hill climbing for 100 points and a  0.7 density is -8 and they were 100000 call to the fitness function.
 The fitness of the initial state is :  (0, 0)
 The fitness at the end of hill climbing for 1000 points and a  0.3 density is -15 and they were 100000 call to the fitness function.
 The fitness of the initial state is :  (0, 0)
 The fitness at the end of hill climbing for 1000 points and a  0.7 density is -15 and they were 100000 call to the fitness function.
 The fitness of the initial state is :  (0, 0)
 The fitness at the end of hill climbing for 5000 points and a  0.3 density is -21 and they were 100000 call to the fitness function.
 The fitness of the initial state is :  (0, 0)


## Simulated annealing